# Motif Scan

In [1]:
from ALLCools.mcds import RegionDS

In [2]:
dmr_ds = RegionDS.open('HIP_small', select_dir=['dmr'])
dmr_ds

Using dmr as region_dim


<xarray.RegionDS>
Dimensions:              (count_type: 2, dmr: 131, sample: 20, sample_collapsed: 10)
Coordinates:
  * count_type           (count_type) <U3 'mc' 'cov'
  * dmr                  (dmr) <U9 'chr1-0' 'chr1-1' ... 'chr19-121' 'chr19-122'
    dmr_chrom            (dmr) <U5 'chr1' 'chr1' 'chr1' ... 'chr19' 'chr19'
    dmr_end              (dmr) int64 10002172 10003542 ... 5099203 5099952
    dmr_length           (dmr) int64 2 305 54 2 2 2 ... 924 632 842 195 399 335
    dmr_ndms             (dmr) int64 1 7 2 1 1 1 2 13 3 ... 2 7 13 19 9 9 3 6 13
    dmr_start            (dmr) int64 10002170 10003237 ... 5098804 5099617
  * sample               (sample) <U18 'snm3C_ASC' 'snm3C_CA1' ... 'snmC_OPC'
  * sample_collapsed     (sample_collapsed) object 'ASC' 'CA1' ... 'ODC' 'OPC'
Data variables:
    dmr_da               (sample, dmr, count_type) uint32 ...
    dmr_da_frac          (sample, dmr) float32 ...
    dmr_state            (sample, dmr) int8 ...
    dmr_state_collapsed  (dmr, sample_collapsed) int8 ...
Attributes:
    chrom_size_path:     /home/hanliu/project/allcools_doc/cluster_level/Regi...
    region_dim:          dmr
    region_ds_location:  /home/hanliu/project/allcools_doc/cluster_level/Regi...

## Default Motif Database

In [3]:
dmr_ds.scan_motifs(genome_fasta='/home/hanliu/ref/mouse/genome/fasta/raw/mm10.fa',
                   cpu=45,
                   standardize_length=500,
                   motif_set_path=None,
                   chrom_size_path=None,
                   combine_cluster=True,
                   chunk_size=10000,
                   dim='motif')

Scan 2179 motif in 131 sequences.
Job 0 returned


In [4]:
dmr_ds

<xarray.RegionDS>
Dimensions:               (count_type: 2, dmr: 131, sample: 20, sample_collapsed: 10, motif: 2179, motif_value: 3, motif-cluster: 286)
Coordinates:
  * count_type            (count_type) <U3 'mc' 'cov'
  * dmr                   (dmr) <U9 'chr1-0' 'chr1-1' ... 'chr19-122'
    dmr_chrom             (dmr) <U5 'chr1' 'chr1' 'chr1' ... 'chr19' 'chr19'
    dmr_end               (dmr) int64 10002172 10003542 ... 5099203 5099952
    dmr_length            (dmr) int64 2 305 54 2 2 2 ... 924 632 842 195 399 335
    dmr_ndms              (dmr) int64 1 7 2 1 1 1 2 13 3 ... 7 13 19 9 9 3 6 13
    dmr_start             (dmr) int64 10002170 10003237 ... 5098804 5099617
  * sample                (sample) <U18 'snm3C_ASC' 'snm3C_CA1' ... 'snmC_OPC'
  * sample_collapsed      (sample_collapsed) object 'ASC' 'CA1' ... 'ODC' 'OPC'
  * motif                 (motif) <U29 'ALX3_homeodomain_2' ... 'ZSC31_HUMAN....
  * motif_value           (motif_value) <U11 'n_motifs' ... 'total_score'
  * motif-cluster         (motif-cluster) object 'c1' 'c10' ... 'c98' 'c99'
Data variables:
    dmr_da                (sample, dmr, count_type) uint32 35 65 506 ... 55 188
    dmr_da_frac           (sample, dmr) float32 0.5385 0.8161 ... 0.2973 0.2926
    dmr_state             (sample, dmr) int8 -1 -1 -1 1 -1 -1 ... 1 1 1 0 0 -1
    dmr_state_collapsed   (dmr, sample_collapsed) int8 0 0 0 0 0 ... 1 1 1 1 -1
    dmr_motif_da          (dmr, motif, motif_value) uint16 ...
    dmr_motif-cluster_da  (motif-cluster, dmr, motif_value) uint16 ...
Attributes:
    chrom_size_path:     /home/hanliu/project/allcools_doc/cluster_level/Regi...
    region_dim:          dmr
    region_ds_location:  /home/hanliu/project/allcools_doc/cluster_level/Regi...